# **Sesión 2:** Procesamiento de Imágenes ⚙️🖼️

## **Instalaciones**

In [2]:
# !pip install scikit-image

## **Librerías**

In [3]:
import os
import cv2
import glob
import imageio
import numpy as np
from typing import List
from utils import non_max_suppression, get_hsv_color_ranges

## **Apartado A: Segmentación por color**

El objetivo de este apartado es segmentar los colores naranja y blanco de las imágenes en la carpeta ``data``.

1. **Tarea A.1**. Defina y ejecute el método para cargar imágenes ``load_imgs()``.
2. **Tarea A.2.** Defina los métodos ``show_image()`` y ``write_image()`` para visualizar y guardar imágenes.
3. **Tarea A.3.** Cambia el espacio de color de las imágenes a uno donde la crominancia y la intensidad estén separados (HSV).
4. **Tarea A.4.** Segmenta los colores anaranjados.
5. **Tarea A.5.** Segmenta los colores blancos.
6. **Tarea A.6.** Junta las máscaras para cada imagen (naranja + blanco) y segmenta cada una de ellas.
7. **Tarea A.7.** Guarda las imágenes.

### **Tarea A.1:** Defina y ejecute el método para cargar imágenes ``load_images()``

In [4]:
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

In [5]:
# TODO Build a list containing the paths of all images in the data folder
imgs_path = [path for path in glob.glob( "data/*jpg" )]
imgs = load_images(imgs_path)

### **Tarea A.2**. Defina los métodos ``show_image()`` y ``write_image()`` para visualizar y guardar imágenes

In [6]:
# TODO Complete the method, use every argument
def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# TODO Complete the method, use every argument
def write_image(output_folder: str, img_name: str, img: np.array):
    img_path = os.path.join(output_folder, img_name)
    cv2.imwrite(img_path, img)

### **Tarea A.3:** Cambie el espacio de color de las imágenes a uno donde la crominancia y la intensidad estén separados (HSV)

In [7]:
# TODO Get a list with ìmgs in HSV color space
hsv_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2HSV) for img in imgs]

# [show_image(img) for img in hsv_imgs]

### **Tarea A.4:** Segmente los colores anaranjados

In [8]:
# TODO Define orange color range
light_orange = (1, 190, 200)
dark_orange = (255, 255, 255)

# TODO Compute a list that contains a mask (which should segment orange colors) for every image.
orange_masks = [cv2.inRange(img, light_orange, dark_orange) for img in hsv_imgs]

# TODO Compute a list that contains the result of multiplying the original image with its orange colors mask.
orange_segmented = [cv2.bitwise_and(img, img, mask = mask) for img,mask in zip(imgs, orange_masks)]

# TODO Show an original image
# [show_image(img) for img in imgs]

# TODO Show a mask
# [show_image(mask) for mask in orange_masks]

# TODO Show a segmented image
# [show_image(img) for img in orange_segmented]


### **Tarea A.5:** Segmente los colores blancos.

Para detectar el rango de blancos complete la siguiente celda y ejecutela para investigar el rango de valores necesarios.

In [9]:
# TODO Discover white color ranges
# get_hsv_color_ranges(imgs[-4]) 

# TODO Define white color range
light_white = (0, 0, 150)
dark_white = (255, 50, 255)

# TODO Compute a list that contains a mask (which should segment white colors) for every image.
white_masks = [cv2.inRange(img, light_white, dark_white) for img in hsv_imgs]

# TODO Compute a list that contains the result of multiplying the original image with its white colors mask.
white_segmented = [cv2.bitwise_and(img, img, mask = mask) for img, mask in zip(imgs, white_masks)]

# TODO Show an original image
# [show_image(img) for img in imgs]

# TODO Show a mask
# [show_image(mask) for mask in white_masks]

# TODO Show a segmented image
# [show_image(img) for img in white_segmented]

### **Tarea A.6:** Junte las máscaras para cada imagen (naranja + blanco) y segmente cada una de ellas.

In [10]:
# TODO Join orange_masks and white_masks
fish_masks = [orange_mask + white_mask for orange_mask, white_mask in zip(orange_masks, white_masks)]
    
# TODO Compute a list that contains the result of multiplying the original image with its complete mask.
fish_images = [cv2.bitwise_and(img, img, mask = mask) for img, mask in zip(imgs, fish_masks)]

# TODO Show an original image
# [show_image(img) for img in imgs]

# TODO Show a mask
# [show_image(mask) for mask in fish_masks]

# TODO Show a segmented image
# [show_image(fish_img) for fish_img in fish_images]


### **Tarea A.7:** Guarde las imágenes

In [11]:
# TODO Define your output folder and save every fish image
output_folder = "processed_data"
os.makedirs(output_folder, exist_ok=True)
save_images = [hsv_imgs, orange_masks, orange_segmented, white_masks, white_segmented, fish_masks, fish_images]
save_names = ["hsv", "orange_mask", "orange_segmented", "white_mask", "white_segmented", "fish_mask", "fish_segmented"]

for count in range(len(imgs)):
    for img_name, img_list in zip(save_names, save_images):
        write_image(output_folder, img_name=img_name+"_"+str(count)+".png", img=img_list[count])


### **Pregunta A.1:** Segmente por color el escudo de su equipo deportivo favorito: descompóngalo en al menos 2 colores. 

In [12]:
# TODO Homework

### **Pregunta A.2:** ¿Qué ocurre si carga las imágenes con la función ``imageio.read()`` y luego la muestra con el método ``show_image()``? ¿A qué se debe este comportamiento?

In [13]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

# TODO Homework: Load images
question_imgs = load_images(None)


# TODO Homework: Show it
show_image(None)

TypeError: 'NoneType' object is not iterable

## **Apartado B:** Filtro Gaussiano y Detección de bordes: Sobel y Canny

El objetivo de este apartado es detectar los bordes de las imágenes de la carpeta ``data``. Para ello, deberá seguir los siguientes pasos:

1. **Tarea B.1.** Defina el método ``gaussian_blur()`` que aplique un filtro gausiano para obtener imágenes borrosas. Siga todas las indicaciones del enunciado.
2. **Tarea B.2.** Aplique el método ``gaussian_blur()`` a todas las imágenes en ``data``.


3. **Tarea B.3.** Defina la función ``sobel_edge_detector()`` que detecte bordes con el método Sobel. Siga todas las indicaciones del enunciado.
4. **Tarea B.4.** Aplique el método ``sobel_edge_detector()`` a todas las imágenes en ``data``.


5. **Tarea B.5.** Defina la función ``canny_edge_detector()`` que detecte bordes con el método Canny. Siga todas las indicaciones del enunciado.
6. **Tarea B.6.** Aplique el método ``canny_edge_detector()`` a todas las imágenes en ``data``.

### **Tarea B.1:** Defina el método ``gaussian_blur()`` que aplique un filtro gausiano para obtener imágenes borrosas.

In [36]:
# TODO Define the method
def gaussian_blur(img: np.array, sigma: float, filter_shape: List | None = None, verbose: bool = False) -> np.array:
    # TODO If not given, compute the filter shape 
    if filter_shape == None:
        filter_l = [-(4*sigma + 1), 4*sigma + 1]
        
    else:
        filter_l = filter_shape
    
    # TODO Create the filter coordinates matrices
    y, x = np.mgrid[filter_l[0]//2 + 1:filter_l[1]//2 + 1, filter_l[0]//2 + 1:filter_l[1]//2 + 1]
    
    # TODO Define the formula that goberns the filter
    gaussian_filter = (1 / (2 * np.pi * sigma**2)) * np.exp(-(x**2 + y**2) / (2 * sigma**2))
    normalized_gaussian_filterr = gaussian_filter/np.sum(gaussian_filter) 

    # TODO Process the image
    gb_img = cv2.filter2D(img, -1, normalized_gaussian_filterr)
    
    if verbose:
        show_image(img=gb_img, img_name=f"Gaussian Blur: Sigma = {sigma}")
    
    return gaussian_filter, gb_img.astype(np.uint8)

### **Tarea B.2.** Aplique el método ``gaussian_blur()`` a todas las imágenes en ``data``.

In [30]:
# TODO Get the gaussian blurred images using a list comprehension
gauss_sigma = 3 # The higher the sigma the blurryer the picture
gb_imgs = [gaussian_blur(img, gauss_sigma, verbose=True)for img in imgs]

### **Tarea B.3:** Defina la función ``sobel_edge_detector()`` que detecte bordes con el método Sobel.

In [15]:
# TODO Define the method
def sobel_edge_detector(img: np.array, filter: np.array, gauss_sigma: float, gauss_filter_shape: List | None = None, verbose: bool = False) -> np.array:
    # TODO Transform the img to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # TODO Get a blurry img to improve edge detections
    blurred = gaussian_blur(img=gray_img, sigma=gauss_sigma, filter_shape=gauss_filter_shape, verbose=verbose)

    # Re-scale
    blurred = blurred[1] / 255
    
    # TODO Get vertical edges
    v_edges = cv2.filter2D(blurred, -1, filter)
    
    # TODO Transform the filter to get the orthogonal edges
    orthogonal_filter = np.array([[row[column] for row in filter] for column in range(len(filter[0]))])

    # TODO Get horizontal edges
    h_edges = cv2.filter2D(blurred, -1, orthogonal_filter)
    
    # TODO Get edges
    sobel_edges_img = np.hypot(v_edges, h_edges)
    
    # Get edges angle
    theta = np.arctan2(v_edges, h_edges)
    
    # Visualize if needed
    if verbose:
        show_image(img=sobel_edges_img, img_name="Sobel Edges")
    
    return np.int8(np.squeeze(sobel_edges_img)), np.int32(np.squeeze(theta))

### **Tarea B.4.** Aplique el método ``sobel_edge_detector()`` a todas las imágenes en ``data``.

In [66]:
# TODO Define a sigma value
gauss_sigma = 1

# TODO Define the Sobel filter
sobel_filter = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

# TODO Get the edges detected by Sobel using a list comprehension
sobel_edges_imgs = [sobel_edge_detector(img, sobel_filter, gauss_sigma, verbose = True)for img in imgs]

### **Tarea B.5:** Defina la función ``canny_edge_detector()`` que detecte bordes con el método Canny.

In [16]:
# TODO Define the method
def canny_edge_detector(img: np.array, sobel_filter: np.array, gauss_sigma: float, gauss_filter_shape: List | None = None, verbose: bool = False):
    # TODO Call the method sobel_edge_detector()
    sobel_edges_img, theta = sobel_edge_detector(img, sobel_filter, gauss_sigma, verbose= True)
    
    # TODO Use NMS to refine edges
    canny_edges_img = non_max_suppression(sobel_edges_img, theta)
    
    if verbose:
        show_image(canny_edges_img, img_name="Canny Edges")
        
    return canny_edges_img

### **Tarea B.6.** Aplique el método ``canny_edge_detector()`` a todas las imágenes en ``data``.

In [28]:
# TODO Define Sobel filter
sobel_filter = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

# TODO Define a sigma value for Gauss
gauss_sigma = 1

# TODO Define a Gauss filter shape
gauss_filter_shape = [-5, 5]

# TODO Get the edges detected by Canny using a list comprehension
canny_edges = [canny_edge_detector(img, sobel_filter, gauss_sigma, verbose = True) for img in imgs]

KeyboardInterrupt: 

### **Pregunta B.1:** Añada ruido a las imágenes de la carpeta ``data``. Compare los resultados que obtiene al aplicar su filtro Sobel con y sin filtro Gausiano.

In [ ]:
# TODO Homework

### **Pregunta B.2:** Utilice la librería ``scikit-image`` y compare el efecto de los filtros Sobel, Canny y Prewitt sobre las imágenes de la carpeta ``data``. ¿Qué diferencias observa entre los filtros? ¿Puede obtener alguna conclusión y/o patrón?

In [ ]:
# TODO Homework

## **Apartado C (Opcional):** Operadores Morfológicos

Para resolver este partado, deberá seguir los siguientes pasos:

1. **Tarea C.1.** Defina el método ``binarize()`` para binarizar imágenes.
2. **Tarea C.2.** Defina el método ``custom_dilate()``.
3. **Tarea C.3.** Defina el método ``custom_erode()``.
4. **Pregunta C.1** Aplique los métodos ``custom_dilate()`` y ``custom_erode()`` a todas las imágenes de la carpeta ``data``.


### **Tarea C.1.** Defina el método ``binarize()`` para binarizar imágenes.

In [46]:
# TODO Homework: define the binarization method
def binarize(img: np.array, mode:str = "Threshold", threshold: int = 127):
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    if mode != "Threshold":
        _, blurry_img = gaussian_blur(gray_img, 1, [-7, 7])
        _, binary_img = cv2.threshold(blurry_img, 0.0, 255.0, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
    else: # Much better work than Otsu
        _, binary_img = cv2.threshold(gray_img, threshold ,255,cv2.THRESH_BINARY)
    
    return binary_img

# [show_image(binarize(img)) for img in imgs]

[None, None, None, None, None, None]

### **Tarea C.2.** Defina el método ``custom_dilate()``

In [60]:
# TODO Homework: define the dilation method
def custom_dilate(img: np.ndarray):
    # TODO pad the original image so it can keep dimensions after processing
    padded = np.pad(img, pad_width=1, mode='constant', constant_values=0)

    # TODO get img shape
    height, width = img.shape

    # TODO Create an element with the same dimensions as the padded img
    dilated = np.zeros_like(padded)

    for j in range(1, height + 1):
        for i in range(1, width + 1):
            # TODO Add logic to the operation
            dilated[j, i] = max([padded[j_p, i_p] for j_p in range(j-1, j+2) for i_p in range(i-1, i+2)])
            
    # TODO Select the region of interest (ROI). Modify if needed
    dilated = dilated[1:height+1, 1:width+1]
    
    return dilated

[show_image(custom_dilate(binarize(img))) for img in imgs]

[None, None, None, None, None, None]

### **Tarea C.3.** Defina el método ``custom_erode()``

In [61]:
# TODO Homework: define the erotion method
def custom_erode(img: np.ndarray):
    # TODO pad the original image so it can keep dimensions after processing
    padded = np.pad(img, pad_width=1, mode='constant', constant_values=255)

    # TODO get img shape
    height, width = img.shape

    # TODO Create an element with the same dimensions as the padded img
    eroded = np.zeros_like(padded)

    for j in range(1, height + 1):
        for i in range(1, width + 1):
            # TODO Add logic to the operation
            eroded[j, i] = min([padded[j_p, i_p] for j_p in range(j-1, j+2) for i_p in range(i-1, i+2)])
            
    # TODO Select the region of interest (ROI). Modify if needed
    eroded = eroded[1:height+1, 1:width+1]
    
    return eroded

[show_image(custom_erode(binarize(img))) for img in imgs]

[None, None, None, None, None, None]

### **Pregunta C.1** Aplique los métodos ``custom_dilate()`` y ``custom_erode()`` a todas las imágenes de la carpeta ``data``.

In [ ]:
# TODO Homework